## 🏗️ Setting Up the Environment 

In [7]:
## Imports needed

from imutils.video import VideoStream
import argparse
import imutils
import time
import cv2
from random import randint
import sys
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
from IPython.display import clear_output
import pickle


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

### 🎥 Setting the path to the video 

In [10]:
PATH_TO_FOLDER = "./"
VIDEO_NAME = "9.avi"

In [11]:
## This cell should run if the video path is read successfully
cap = cv2.VideoCapture(PATH_TO_FOLDER + VIDEO_NAME)

### ⚙️ Creating the Object Used to Save Labels 

In [21]:
class AgentSaver:   
    
    
    def __init__(self):
        self.agents = []
        self.bboxes = []
        self.colors = []
        self.multiTracker = cv2.MultiTracker_create()
        self.trackerType = "csrt" 
        
        self.index_of_agents = {}
        self.label_of_agents = {}
        
        self.labeleddata = {}
        
        self.maxAgentNo = 1
        
    def prepareForSaving(self):
        self.multiTracker = None
    
    def reviveFromSavedState(self):
        self.multiTracker = cv2.MultiTracker_create()
        
        for bbox in self.bboxes:
              self.multiTracker.add(OPENCV_OBJECT_TRACKERS[self.trackerType](), frame, bbox)
                    
    
    def addAgent(self, box, color, agentNo, typeOfAgent):
        index = self.index_of_agents.get(agentNo, -1)
        
        if index == -1:
            
            self.index_of_agents[agentNo] = len(self.agents)
            self.label_of_agents[agentNo] = typeOfAgent
            
            self.agents.append(agentNo)
            self.bboxes.append(box)
            
            self.multiTracker.add(OPENCV_OBJECT_TRACKERS[self.trackerType](), frame, box)
            self.colors.append(color)
            
        else:
            self.bboxes[index] = box
            self.multiTracker = cv2.MultiTracker_create()
            
            for bbox in self.bboxes:
                  self.multiTracker.add(OPENCV_OBJECT_TRACKERS[self.trackerType](), frame, bbox)
            
    
    def removeAgentByNumber(self, agentNo):
        index = self.index_of_agents.get(agentNo, -1)
        
        if index == -1:
            return
        
        self.bboxes.pop(index)
        self.colors.pop(index)
        self.agents.pop(index)
        
        self.index_of_agents.pop(agentNo)
        self.label_of_agents.pop(agentNo)
        
        self.multiTracker = cv2.MultiTracker_create()            
        for bbox in self.bboxes:
              self.multiTracker.add(OPENCV_OBJECT_TRACKERS[self.trackerType](), frame, bbox)
        
        self.resetIndices(index)
    
    def removeAgentByIndex(self, index):
        agentNo = self.agents[index]
        
        self.bboxes.pop(index)
        self.colors.pop(index)
        self.agents.pop(index)
        
        self.index_of_agents.pop(agentNo)
        self.label_of_agents.pop(agentNo)
        
        self.multiTracker = cv2.MultiTracker_create()            
        for bbox in self.bboxes:
              self.multiTracker.add(OPENCV_OBJECT_TRACKERS[self.trackerType](), frame, bbox)
                    
        self.resetIndices(index)
            
    def resetIndices(self, index):
        for agentNo, oldIndex in self.index_of_agents.items():
            if oldIndex > index:
                self.index_of_agents[agentNo] = oldIndex - 1
        
    def refreshBoxes(self):
        self.bboxes = [(*list(x),) for x in agentSaver.multiTracker.getObjects()]
    
    
    def delete_model_output(self, frameno, object_id):
        title = 'left' + str(frameno).zfill(6) + '.png'
        notcount = 0
        while title in self.labeleddata.keys():
            if object_id not in self.labeleddata[title].keys():
                notcount = notcount + 1
                if notcount == 2:
                    break
                else:
                    continue
            
            self.labeleddata[title].pop(object_id)
            frameno = frameno + 1
            title = 'left' + str(frameno).zfill(6) + '.png'
    
   
        
    def saveframedata(self, frameno):
        title = 'left' + str(frameno).zfill(6) + '.png'
        
        agentsboxes = {}
        
        for (agent, box) in zip(self.agents, self.bboxes):
            x1 = int(box[0])
            y1 = int(box[1])
            x2 = int(box[0] + box[2])
            y2 = int(box[1] + box[3])
            agentsboxes[agent] = ([x1, y1, x2, y2], self.label_of_agents[agent])
        
        self.labeleddata[title] = agentsboxes
        
    def savedetections(self, frameno, detections):
        title = str(frameno).zfill(6)
        
        agentsboxes = {}
        
        for (box, agent, label) in detections:
            x1 = int(box[0])
            y1 = int(box[1])
            x2 = int(box[2])
            y2 = int(box[3])
            
            agentsboxes[-agent] = ([x1, y1, x2, y2], label)
        
        self.labeleddata[title] = agentsboxes

## 🏷️ Label Bounding Boxes Using CV2 Tracker

In [9]:
OPENCV_OBJECT_TRACKERS = {
    "csrt": cv2.TrackerCSRT_create,
    "kcf": cv2.TrackerKCF_create,
    "boosting": cv2.TrackerBoosting_create,
    "mil": cv2.TrackerMIL_create,
    "tld": cv2.TrackerTLD_create,
    "medianflow": cv2.TrackerMedianFlow_create,
    "mosse": cv2.TrackerMOSSE_create
}

trackers = cv2.MultiTracker_create()

In [12]:
cap = cv2.VideoCapture(PATH_TO_FOLDER + VIDEO_NAME)

In [13]:
success, frame = cap.read()

### ✏️ Drawing the initial bounding boxes

In [14]:
agentcount = 1

agentSaver = AgentSaver()
 
# OpenCV's selectROI function doesn't work for selecting multiple objects in Python
# So we will call this function in a loop till we are done selecting all objects
while True:
    # draw bounding boxes over objects
    # selectROI's default behaviour is to draw box starting from the center
    # when fromCenter is set to false, you can draw box starting from top left corner
    print('\n\n')
    
    bbox = cv2.selectROI('MultiTracker', frame)
    color = (randint(100, 200), randint(100, 200), randint(50, 150))
    
    type_of_agent = "Car"
    choice = input("What is the type of agent?\nh Human\nr Ricksaw\na Animal\nm Motorcycle\nc Car\nt Truck\nb Bicycle")
    if choice == 'h':
        type_of_agent = "Human"
    elif choice == 'r':
        type_of_agent = 'Ricksaw'
    elif choice == 'a':
        type_of_agent = 'Animal'
    elif choice == 'b':
        type_of_agent = 'Bicycle'
    elif choice == 'm':
        type_of_agent = 'Motorcycle'
    elif choice == 't':
        type_of_agent = 'Truck'
        
    
    agentSaver.addAgent(bbox, color, agentcount, type_of_agent)


    print("Adding as agent ", agentcount," which is ", type_of_agent, "\nAdd more? y n")

    k = input("label more?")
    agentcount = agentcount + 1
    if (k == 'n'):  # q is pressed
        break




What is the type of agent?
h Human
r Ricksaw
a Animal
m Motorcycle
c Car
t Truck
b Bicycleb
Adding as agent  1  which is  Bicycle 
Add more? y n
label more?



What is the type of agent?
h Human
r Ricksaw
a Animal
m Motorcycle
c Car
t Truck
b Bicycley
Adding as agent  2  which is  Car 
Add more? y n
label more?n


In [22]:
## Setting the limit to make sure boxes going outside this limit are deleted automtically
RIGHT_LIMIT = 640
BOTTOM_LIMIT = 475

### 💾 Helper Functions to Save and Load Agent in between labeling

In [16]:
def saveAgent(frameno):
    agentSaver.maxAgentNo = agentcount
    agentSaver.prepareForSaving()
    with open('agent_saver_{}_{}.pkl'.format(1, frameno - 1), 'wb') as config_dictionary_file:
        pickle.dump(agentSaver, config_dictionary_file)

In [19]:
def loadFrameNo(frameNo, continueMode = False):
    global frame, agentSaver, cap, frameno, agentcount
    
    cap = cv2.VideoCapture(PATH_TO_FOLDER + VIDEO_NAME)
    
    if continueMode:
        for i in range(frameNo + 1):
            success, frame = cap.read()

    with open('agent_saver_{}_{}.pkl'.format(1, frameNo), 'rb') as config_dictionary_file:
        agentSaver = pickle.load(config_dictionary_file)
    
    
    frameno = frameNo
    agentcount = agentSaver.maxAgentNo
    
    agentSaver.reviveFromSavedState()

### 📝 Load the last labeled state and continue labeling

In [18]:
LAST_LABELED_FRAME_NO = 1324

loadFrameNo(LAST_LABELED_FRAME_NO, continueMode=True)

In [20]:
# Process video and track objects
frame_id = frameno
while cap.isOpened():
    global frame
    
    clear_output()
    frame_id = frame_id + 1
    success, frame = cap.read()

    if not success:
        break

    save = False
    
    # get updated location of objects in subsequent frames
    
    success, boxes = agentSaver.multiTracker.update(frame)
    
    agentSaver.refreshBoxes()
    
    to_remove_agents = []
    
    # draw tracked objects
    for i, newbox in enumerate(boxes):
        x1 = int(newbox[0])
        y1 = int(newbox[1])
        x2 = int(newbox[0] + newbox[2])
        y2 = int(newbox[1] + newbox[3])
        if x1 < RIGHT_LIMIT and x1 >= 0 and y1 >= 0:
            if x2 < RIGHT_LIMIT and y2 < BOTTOM_LIMIT:
                p1 = (x1, y1)
                p2 = (x2, y2)
                cv2.rectangle(frame, p1, p2, agentSaver.colors[i], 2, 1)
                cv2.putText(frame,'{}'.format(agentSaver.agents[i]), (x1,y1), cv2.FONT_HERSHEY_SIMPLEX, 1, agentSaver.colors[i])
            else:
                to_remove_agents.append(i)
        
        else:
            to_remove_agents.append(i)
    
    
    if to_remove_agents != None:
        for num in to_remove_agents[::-1]:
            agentSaver.removeAgentByIndex(num)

        
    # show frame    
    cv2.imshow('MultiTracker', frame)
    
    c = (cv2.waitKey(1) & 0xFF); 
    while c != 110 and c!=121 and (c != 27):
        c = (cv2.waitKey(1) & 0xFF);
        continue
    

    if c==121:
        while True:
            bbox = cv2.selectROI('MultiTracker', frame)
            color = (randint(100, 200), randint(100, 200), randint(50, 150))
    
                
    
            print("Adding as agent ", agentcount,"\nAdd more? y n e d q s")

            k = input("label more?")
            if (k == 'n'): 
                type_of_agent = "Car"
                choice = input("What is the type of agent?\nh Human\nr Ricksaw\na Animal\nm Motorcycle\nc Car\nt Truck\nb Bicycle")
                if choice == 'h':
                    type_of_agent = "Human"
                elif choice == 'r':
                    type_of_agent = 'Ricksaw'
                elif choice == 'a':
                    type_of_agent = 'Animal'
                elif choice == 'b':
                    type_of_agent = 'Bicycle'
                elif choice == 'm':
                    type_of_agent = 'Motorcycle'
                elif choice == 't':
                    type_of_agent = 'Truck'
                
                agentSaver.addAgent(bbox, color, agentcount, type_of_agent)
                agentcount = agentcount + 1
                break
                
            elif k == 'e':
                no = int(input("Enter agent number?"))
                agentSaver.addAgent(bbox, color, no, 'car')
            
            elif k == 'd':
                no = int(input("Enter agent number?"))
                agentSaver.removeAgentByNumber(no)
                
                
            elif k == 'q':
                break
                
            elif k == 's':
                save = True
                break
                
            else:
                type_of_agent = "Car"
                choice = input("What is the type of agent?\nh Human\nr Ricksaw\na Animal\nm Motorcycle\nc Car\nt Truck\nb Bicycle")
                if choice == 'h':
                    type_of_agent = "Human"
                elif choice == 'r':
                    type_of_agent = 'Ricksaw'
                elif choice == 'a':
                    type_ofy_agent = 'Animal'
                elif choice == 'b':
                    type_of_agent = 'Bicycle'
                elif choice == 'm':
                    type_of_agent = 'Motorcycle'
                elif choice == 't':
                    type_of_agent = 'Truck'
                    
                agentSaver.addAgent(bbox, color, agentcount, type_of_agent)
                agentcount = agentcount + 1
                
            

    agentSaver.saveframedata(-frame_id)
    
    if save == True:
        saveAgent()
        agentSaver.reviveFromSavedState()
        
    # quit on ESC button 
    if c & 0xFF == 27:  # Esc pressed
        break